<a href="https://colab.research.google.com/github/sabinagio/what-s-the-matter/blob/main/twitter_api_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter API data collection - proof of concept

In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

## 1. Define data collection functions

In [2]:
# To be used once then removed
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAFwFgQEAAAAAOjRNg8PpoZ29uTo2Z%2By0K7DEM3s%3DI1NuBq4xOCsYtDMg8nos58Io1kQKgdW80FSEKaaFcSK69TueFy'

In [3]:
# Authenticate with the bearer token
def auth():
    return os.getenv('TOKEN')

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [5]:
def create_url(keyword, max_results=100):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent"
    query_params = {'query': keyword, 
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id', 
                    'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld', 
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified', 
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type', 
                    'next_token': {}}
    return (search_url, query_params)

In [6]:
def connect_to_endpoint(url, headers, params, next_token=None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers=headers, params=params)
    print("Endpoint Response Code: " + str(response.status_code))

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    return response.json()

## 2. Explore Twitter API request response

In [7]:
# Create first API request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = '''fear OR fearful OR afraid OR scared OR terrified OR worry OR worried OR anxiety OR anxious \
  OR distress OR concern OR dismay OR strain OR stress OR tension -"nothing to fear" -"fear not" -"don't worry" -"no worries" lang:en'''

In [8]:
url = create_url(keyword)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [9]:
json_response

{'data': [{'public_metrics': {'retweet_count': 6,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'author_id': '1099865044223877120',
   'created_at': '2022-08-27T05:40:38.000Z',
   'conversation_id': '1563401081094545409',
   'source': 'Twitter for Android',
   'possibly_sensitive': False,
   'reply_settings': 'everyone',
   'entities': {'mentions': [{'start': 3,
      'end': 11,
      'username': 'jhnnjsh',
      'id': '1322635027545350144'}]},
   'text': 'RT @jhnnjsh: [ MINI GIVEAWAY ]\nKarena lagi stress kerja, aku mau bagi bagi saldo DANA 10k buat 1 orang. \n\nSyaratnya rt ajadehhh, kalau mau…',
   'id': '1563401081094545409',
   'referenced_tweets': [{'type': 'retweeted', 'id': '1563399866847760384'}]},
  {'public_metrics': {'retweet_count': 0,
    'reply_count': 0,
    'like_count': 0,
    'quote_count': 0},
   'author_id': '1611978637',
   'created_at': '2022-08-27T05:40:38.000Z',
   'context_annotations': [{'domain': {'id': '10',
      'name': 'Person',
  

In [10]:
json_response.keys()

dict_keys(['data', 'includes', 'meta'])

In [17]:
json_response["data"][1].keys()

dict_keys(['public_metrics', 'author_id', 'created_at', 'context_annotations', 'conversation_id', 'source', 'possibly_sensitive', 'reply_settings', 'entities', 'in_reply_to_user_id', 'text', 'id', 'referenced_tweets'])

In [18]:
json_response["data"][1]["public_metrics"]

{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}

In [13]:
len(json_response["data"])

10

In [19]:
json_response["data"][1]

{'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'author_id': '1611978637',
 'created_at': '2022-08-27T05:40:38.000Z',
 'context_annotations': [{'domain': {'id': '10',
    'name': 'Person',
    'description': 'Named people in the world like Nelson Mandela'},
   'entity': {'id': '940653064968531968',
    'name': 'Andy Biggs',
    'description': 'US Representative Andy Biggs (AZ-5)'}},
  {'domain': {'id': '35',
    'name': 'Politician',
    'description': 'Politicians in the world, like Joe Biden'},
   'entity': {'id': '940653064968531968',
    'name': 'Andy Biggs',
    'description': 'US Representative Andy Biggs (AZ-5)'}},
  {'domain': {'id': '131',
    'name': 'Unified Twitter Taxonomy',
    'description': 'A taxonomy view into the Semantic Core knowledge graph'},
   'entity': {'id': '847878884917886977',
    'name': 'Politics',
    'description': 'Politics'}},
  {'domain': {'id': '131',
    'name': 'Unified Twitter Taxonomy',
    'd

> Entity annotations (NER): Entities are comprised of people, places, products, and organizations. Entities are delivered as part of the entity payload section. They are programmatically assigned based on what is explicitly mentioned (named-entity recognition) in the Tweet text.

> Entity annotations are programmatically defined entities that are nested within the entities field and are reflected as annotations in the payload. Each annotation has a confidence score and an indication of where in the Tweet text the entities were identified (start and end fields).

> The entity annotations can have the following types:
> - Person - Barack Obama, Daniel, or George W. Bush
> - Place - Detroit, Cali, or "San Francisco, California"
> - Product - Mountain Dew, Mozilla Firefox
> - Organization - Chicago White Sox, IBM
> - Other - Diabetes, Super Bowl 50

In [20]:
json_response["data"][1]["entities"]

{'mentions': [{'start': 0,
   'end': 15,
   'username': 'RepAndyBiggsAZ',
   'id': '816652616625168388'},
  {'start': 16, 'end': 31, 'username': 'TheJusticeDept', 'id': '73181712'}]}

> Context annotations: Derived from the analysis of a Tweet’s text and will include a domain and entity pairing which can be used to discover Tweets on topics that may have been previously difficult to surface. At present, we’re using a list of 80+ domains to categorize Tweets. A CSV file of the available context annotation entities is available for download at our [Github repository](https://github.com/twitterdev/twitter-context-annotations).

In [21]:
json_response["data"][1]["context_annotations"]

[{'domain': {'id': '10',
   'name': 'Person',
   'description': 'Named people in the world like Nelson Mandela'},
  'entity': {'id': '940653064968531968',
   'name': 'Andy Biggs',
   'description': 'US Representative Andy Biggs (AZ-5)'}},
 {'domain': {'id': '35',
   'name': 'Politician',
   'description': 'Politicians in the world, like Joe Biden'},
  'entity': {'id': '940653064968531968',
   'name': 'Andy Biggs',
   'description': 'US Representative Andy Biggs (AZ-5)'}},
 {'domain': {'id': '131',
   'name': 'Unified Twitter Taxonomy',
   'description': 'A taxonomy view into the Semantic Core knowledge graph'},
  'entity': {'id': '847878884917886977',
   'name': 'Politics',
   'description': 'Politics'}},
 {'domain': {'id': '131',
   'name': 'Unified Twitter Taxonomy',
   'description': 'A taxonomy view into the Semantic Core knowledge graph'},
  'entity': {'id': '940653064968531968',
   'name': 'Andy Biggs',
   'description': 'US Representative Andy Biggs (AZ-5)'}},
 {'domain': {'id':

In [22]:
json_response["includes"]['users'][1].keys()

dict_keys(['public_metrics', 'username', 'id', 'description', 'verified', 'name', 'created_at'])

In [23]:
json_response["includes"]['users'][1]

{'public_metrics': {'followers_count': 3154,
  'following_count': 3912,
  'tweet_count': 17609,
  'listed_count': 4},
 'username': 'GoBigRedResist',
 'id': '1611978637',
 'description': 'Faith, Democracy, and Huskers. 🏈 GBR 🎈Occasionally I share bad jokes.',
 'verified': False,
 'name': '🌽HuskerHawk🌽',
 'created_at': '2013-07-22T03:51:03.000Z'}

In [59]:
json_response["includes"]['users'][1]["public_metrics"]

{'followers_count': 3154,
 'following_count': 3912,
 'tweet_count': 17609,
 'listed_count': 4}

In [24]:
len(json_response["includes"]['users'])

11

In [25]:
json_response["meta"].keys()

dict_keys(['newest_id', 'oldest_id', 'result_count', 'next_token'])

In [26]:
json_response["meta"]

{'newest_id': '1563401081094545409',
 'oldest_id': '1563401077432520704',
 'result_count': 10,
 'next_token': 'b26v89c19zqg8o3fpz8l5nshu7p9lzg9gulsc2yorsepp'}

## 3. Convert JSON response to a pandas dataframe for further analysis

In [60]:
# Create empty dataframes for tweet & user data
tweet_cols = ['possibly_sensitive', 'created_at', 'entities', 'context_annotations', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'id', 'referenced_tweets', \
              'conversation_id', 'in_reply_to_user_id', 'author_id', 'source', 'text', 'reply_settings']
tweets_data = pd.DataFrame(columns=tweet_cols)

user_cols = ['followers_count', 'following_count', 'tweet_count', 'listed_count', 'username', 'id', 'description', 'verified', 'name', 'created_at']
users_data = pd.DataFrame(columns=user_cols)

In [61]:
def extract_data(json_response, dataframe, data_type):
  data_dict = {"tweet": json_response["data"], "user": json_response["includes"]["users"]}
  data_source = data_dict[data_type]
  for row in data_source:

    # Extract the public metrics data
    new_row = row.copy()
    public_metrics = new_row["public_metrics"].copy()
    for key in public_metrics.keys():
      new_row[key] = public_metrics[key]
    new_row.pop("public_metrics")

    # Add tweet/user data to dataframe with unpacked public metrics
    dataframe = dataframe.append(new_row, ignore_index=True)
  return dataframe

In [53]:
tweets_data = extract_data(json_response, tweets_data, data_type="tweet")

In [54]:
# Check that data is correctly appended at the end of the dataframe
tweets_data.shape

(30, 17)

In [55]:
tweets_data

,possibly_sensitive,created_at,entities,context_annotations,retweet_count,reply_count,like_count,quote_count,id,referenced_tweets,conversation_id,in_reply_to_user_id,author_id,source,text,reply_settings,geo
0,False,2022-08-27T05:40:38.000Z,"{'mentions': [{'start': 3, 'end': 11, 'usernam...",NaN,6,0,0,0,1563401081094545409,"[{'type': 'retweeted', 'id': '1563399866847760...",1563401081094545409,NaN,1099865044223877120,Twitter for Android,RT @jhnnjsh: [ MINI GIVEAWAY ]\nKarena lagi st...,everyone,NaN
1,False,2022-08-27T05:40:38.000Z,"{'mentions': [{'start': 0, 'end': 15, 'usernam...","[{'domain': {'id': '10', 'name': 'Person', 'de...",0,0,0,0,1563401080523739137,"[{'type': 'replied_to', 'id': '156320062547456...",1563200625474564096,816652616625168388,1611978637,Twitter for iPhone,@RepAndyBiggsAZ @TheJusticeDept Worried bro?,everyone,NaN
2,False,2022-08-27T05:40:38.000Z,"{'mentions': [{'start': 3, 'end': 14, 'usernam...","[{'domain': {'id': '10', 'name': 'Person', 'de...",1457,0,0,0,1563401080066945024,"[{'type': 'retweeted', 'id': '1563204001323110...",1563401080066945024,NaN,1489788293600497664,Twitter for iPhone,RT @ForAmerica: ⬛⬛⬛⬛⬛Joe⬛⬛⬛⬛⬛⬛Biden's⬛⬛⬛⬛⬛⬛FBI...,everyone,NaN
3,False,2022-08-27T05:40:38.000Z,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",NaN,131,0,0,0,1563401079223566337,"[{'type': 'retweeted', 'id': '1563123281502167...",1563401079223566337,NaN,3124523661,Twitter for Android,"RT @QuranAndHadithh: ""I fear the day when the ...",everyone,NaN
4,False,2022-08-27T05:40:38.000Z,NaN,NaN,0,0,0,0,1563401078976434176,NaN,1563401078976434176,NaN,891168452370616321,Twitter for Android,m*dd*x tak letak catalogue awal2 and i just sa...,following,NaN
5,False,2022-08-27T05:40:38.000Z,"{'mentions': [{'start': 3, 'end': 18, 'usernam...","[{'domain': {'id': '159', 'name': 'States', 'd...",338,0,0,0,1563401078888345600,"[{'type': 'retweeted', 'id': '1563014568007282...",1563401078888345600,NaN,91797997,Twitter for Android,RT @RahulMukherji5: This must feel horrible &a...,everyone,NaN
6,False,2022-08-27T05:40:38.000Z,NaN,NaN,0,0,0,0,1563401078791475200,NaN,1563401078791475200,NaN,814353358593007616,Twitter for iPhone,Feel Fear and still do it,everyone,{'place_id': 'a612c69b44b2e5da'}
7,False,2022-08-27T05:40:38.000Z,NaN,NaN,0,0,0,0,1563401078606946304,NaN,1563401078606946304,NaN,1792734204,Echofon Android,passion androgynous anxiety broken linear\n؟?\...,everyone,NaN
8,False,2022-08-27T05:40:38.000Z,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",NaN,106,0,0,0,1563401077621653504,"[{'type': 'retweeted', 'id': '1563353223712219...",1563401077621653504,NaN,1392623516869177350,Twitter for iPhone,"RT @TomJumboGrumbo: Drip #150348, a puppy who ...",everyone,NaN
9,False,2022-08-27T05:40:38.000Z,"{'urls': [{'start': 69, 'end': 92, 'url': 'htt...","[{'domain': {'id': '10', 'name': 'Person', 'de...",59,0,0,0,1563401077432520704,"[{'type': 'retweeted', 'id': '1563085089239859...",1563401077432520704,NaN,1476515154607034369,Twitter for Android,RT @loeymarie: predestined to be the biggest l...,everyone,NaN


In [65]:
users_data = extract_data(json_response, users_data, data_type="user")

In [66]:
users_data.shape

(22, 10)

In [64]:
users_data

,followers_count,following_count,tweet_count,listed_count,username,id,description,verified,name,created_at
0,24,392,4926,0,zae12345678,1099865044223877120,Hidup yang tidak dipertaruhkan tidak akan pern...,False,@zae,2019-02-25T02:53:55.000Z
1,3154,3912,17609,4,GoBigRedResist,1611978637,"Faith, Democracy, and Huskers. 🏈 GBR 🎈Occasion...",False,🌽HuskerHawk🌽,2013-07-22T03:51:03.000Z
2,488237,1757,21713,2103,RepAndyBiggsAZ,816652616625168388,Representing Arizona's Fighting Fifth.,True,Rep Andy Biggs,2017-01-04T14:28:51.000Z
3,239,636,4401,0,LookUp_OK,1489788293600497664,I am a believer in free speech and enjoy open ...,False,K Ok,2022-02-05T02:30:08.000Z
4,22,122,3726,0,Ishaq1987,3124523661,,False,Ishaq Shaikh,2015-03-28T19:11:52.000Z
5,495,3661,76071,10,vanessa_kim5,891168452370616321,Personal ramblings. Unapologetically jumps fan...,False,naz // thessum spoilers in likes,2017-07-29T05:28:10.000Z
6,418,268,15366,1,garv5137,91797997,"poetry, humanity, kindness, zindagii ❤️\n\nhtt...",False,Gaurav Sinha 🇮🇳,2009-11-22T14:23:50.000Z
7,171,170,2007,0,GhettoRichBoy,814353358593007616,🔮,False,brimreaper,2016-12-29T06:12:25.000Z
8,1,12,178,0,agogo2080,1792734204,Abolish Ability Accelerators Absolute About ac...,False,ك̲ۣو̲ۣد̲ۣ ك̲ۣو̲ب̲و̲ۣن̲ خ̲ۣص̲ۣم̲,2013-09-08T00:56:04.000Z
9,231,326,23916,1,Jeannie1504,1392623516869177350,,False,Jeannie W,2021-05-12T23:31:54.000Z


## 4. Collect 10,000 tweets for initial exploration & analysis

In [94]:
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = '''fear OR fearful OR afraid OR scared OR terrified OR worry OR worried OR anxiety OR anxious \
  OR distress OR concern OR dismay OR strain OR stress OR tension -"nothing to fear" -"fear not" -"don't worry" -"no worries" lang:en'''
url = create_url(keyword)

# Create empty dataframes for tweet & user data
tweet_cols = ['possibly_sensitive', 'created_at', 'entities', 'context_annotations', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'id', 'referenced_tweets', \
              'conversation_id', 'in_reply_to_user_id', 'author_id', 'source', 'text', 'reply_settings']
tweets_data = pd.DataFrame(columns=tweet_cols)

user_cols = ['followers_count', 'following_count', 'tweet_count', 'listed_count', 'username', 'id', 'description', 'verified', 'name', 'created_at']
users_data = pd.DataFrame(columns=user_cols)

flag = True
next_token = None

# Attempt to get all of the tweets that match the query and save into a dataframe 
while flag:
  json_response = connect_to_endpoint(url[0], headers, url[1], next_token=next_token)
  next_token = json_response["meta"]["next_token"]
  tweets_data = extract_data(json_response, tweets_data, data_type="tweet")
  users_data = extract_data(json_response, users_data, data_type="user")
  print("Collected {} tweets from {} users!".format(tweets_data.shape[0], users_data.shape[0]))

  # Stop the loop if it collected 10,000 tweets for initial analysis
  if tweets_data.shape[0] >= 10000:
    break

  if next_token == None:
    flag = False

  # Add some time between API requests so the system doesn't restrict access
  time.sleep(5)

Endpoint Response Code: 200
Collected 10 tweets from 10 users!
Endpoint Response Code: 200
Collected 20 tweets from 23 users!
Endpoint Response Code: 200
Collected 30 tweets from 34 users!
Endpoint Response Code: 200
Collected 40 tweets from 46 users!
Endpoint Response Code: 200
Collected 50 tweets from 56 users!
Endpoint Response Code: 200
Collected 60 tweets from 68 users!
Endpoint Response Code: 200
Collected 70 tweets from 80 users!
Endpoint Response Code: 200
Collected 80 tweets from 93 users!
Endpoint Response Code: 200
Collected 90 tweets from 105 users!
Endpoint Response Code: 200
Collected 100 tweets from 118 users!
Endpoint Response Code: 200
Collected 110 tweets from 130 users!
Endpoint Response Code: 200
Collected 120 tweets from 140 users!
Endpoint Response Code: 200
Collected 130 tweets from 153 users!
Endpoint Response Code: 200
Collected 140 tweets from 164 users!
Endpoint Response Code: 200
Collected 149 tweets from 175 users!
Endpoint Response Code: 200
Collected 159 

In [95]:
# Check that the tweets extracted are unique
len(tweets_data.id.unique())

10003

In [96]:
tweets_data.head(20)

,possibly_sensitive,created_at,entities,context_annotations,retweet_count,reply_count,like_count,quote_count,id,referenced_tweets,conversation_id,in_reply_to_user_id,author_id,source,text,reply_settings,attachments,geo,withheld
0,False,2022-08-27T07:28:07.000Z,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",NaN,77,0,0,0,1563428129980690434,"[{'type': 'retweeted', 'id': '1563426879419580...",1563428129980690434,NaN,141925744,Twitter for Android,"RT @firstladyship: My People, our Kaduna rally...",everyone,NaN,NaN,NaN
1,False,2022-08-27T07:28:07.000Z,"{'urls': [{'start': 64, 'end': 87, 'url': 'htt...",NaN,0,0,0,0,1563428129829703682,NaN,1563428129829703682,NaN,1309757846335225856,Twitter for iPhone,"“Anxiety weighs down the heart, but a kind wor...",everyone,NaN,NaN,NaN
2,False,2022-08-27T07:28:07.000Z,NaN,NaN,0,0,0,0,1563428129288306688,NaN,1563428129288306688,NaN,557697314,Twitter for iPhone,This GIGGO app dey stress my life.,everyone,NaN,NaN,NaN
3,False,2022-08-27T07:28:07.000Z,"{'mentions': [{'start': 3, 'end': 15, 'usernam...","[{'domain': {'id': '10', 'name': 'Person', 'de...",16170,0,0,0,1563428128818573312,"[{'type': 'retweeted', 'id': '1563224720085921...",1563428128818573312,NaN,3238093166,Twitter Web App,RT @Jack_Petocz: Ron DeSantis just removed 4 d...,everyone,NaN,NaN,NaN
4,False,2022-08-27T07:28:07.000Z,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",NaN,1,0,0,0,1563428128520687616,"[{'type': 'retweeted', 'id': '1563427342617169...",1563428128520687616,NaN,1457998353946406913,Twitter for Android,RT @martinezeunicec: Having therefore these pr...,everyone,NaN,NaN,NaN
5,False,2022-08-27T07:28:07.000Z,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",NaN,760,0,0,0,1563428128415854592,"[{'type': 'retweeted', 'id': '1562914241786433...",1563428128415854592,NaN,2170782053,Twitter for iPhone,RT @Cianniambrose: POV: you finally met a man ...,everyone,NaN,NaN,NaN
6,False,2022-08-27T07:28:07.000Z,"{'urls': [{'start': 108, 'end': 131, 'url': 'h...",NaN,0,0,0,0,1563428127610531843,"[{'type': 'quoted', 'id': '1563426615580733440'}]",1563428127610531843,NaN,1213876078634098688,Twitter for Android,"wala na kong hahalikan kada naalis ako, don't ...",everyone,NaN,NaN,NaN
7,True,2022-08-27T07:28:07.000Z,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",NaN,2,0,0,0,1563428127363084293,"[{'type': 'retweeted', 'id': '1563414961732210...",1563428127363084293,NaN,1562671544601497600,Twitter Web App,RT @ja_armitxe: Don't be afraid https://t.co/Z...,everyone,{'media_keys': ['7_1563414879716794369']},NaN,NaN
8,False,2022-08-27T07:28:07.000Z,"{'mentions': [{'start': 3, 'end': 18, 'usernam...",NaN,1,0,0,0,1563428127078252544,"[{'type': 'retweeted', 'id': '1563421374764371...",1563428127078252544,NaN,1209865490,Twitter Web App,RT @ShropshireStar: Government action is neede...,everyone,NaN,NaN,NaN
9,False,2022-08-27T07:28:06.000Z,"{'mentions': [{'start': 3, 'end': 12, 'usernam...",NaN,1374,0,0,0,1563428125169442818,"[{'type': 'retweeted', 'id': '1563371399212699...",1563428125169442818,NaN,1468218581905448966,Twitter for Android,RT @yunjikeu: Jake mentioned before that he’s ...,everyone,NaN,NaN,NaN


In [97]:
tweets_data.tail(20)

,possibly_sensitive,created_at,entities,context_annotations,retweet_count,reply_count,like_count,quote_count,id,referenced_tweets,conversation_id,in_reply_to_user_id,author_id,source,text,reply_settings,attachments,geo,withheld
9983,False,2022-08-27T07:07:56.000Z,"{'hashtags': [{'start': 33, 'end': 38, 'tag': ...",NaN,10,0,0,0,1563423050426884096,"[{'type': 'retweeted', 'id': '1563419946683858...",1563423050426884096,NaN,7797552,Twitter for Android,"RT @delaneykingrox: Okay another #adhd thread,...",everyone,NaN,NaN,NaN
9984,False,2022-08-27T07:07:56.000Z,"{'mentions': [{'start': 0, 'end': 8, 'username...","[{'domain': {'id': '67', 'name': 'Interests an...",0,0,1,0,1563423049797758976,"[{'type': 'replied_to', 'id': '156342176952002...",1563411964415463424,2975225778,1394628330872348682,Twitter Web App,"@AwTroon well yes, but using a modern farm mac...",everyone,NaN,NaN,NaN
9985,False,2022-08-27T07:07:56.000Z,"{'hashtags': [{'start': 184, 'end': 199, 'tag'...",NaN,0,1,0,0,1563423049235705856,NaN,1563423049235705856,NaN,1563367576641896452,Twitter for Android,Im super worried about jake i know she fell be...,everyone,NaN,NaN,NaN
9986,False,2022-08-27T07:07:56.000Z,"{'hashtags': [{'start': 93, 'end': 102, 'tag':...","[{'domain': {'id': '66', 'name': 'Interests an...",1,0,0,0,1563423048925007874,"[{'type': 'retweeted', 'id': '1563422745232547...",1563423048925007874,NaN,748364168,Twitter for Android,RT @marginalheroes: 【宣伝】『チェンジアクションRPG マージナルヒーロ...,everyone,NaN,NaN,NaN
9987,False,2022-08-27T07:07:56.000Z,"{'annotations': [{'start': 93, 'end': 98, 'pro...",NaN,1234,0,0,0,1563423048883404800,"[{'type': 'retweeted', 'id': '1563283749202042...",1563423048883404800,NaN,839606118175420416,Twitter Web App,RT @ChudeMedia: Why did Governor @elrufai gave...,everyone,NaN,NaN,NaN
9988,False,2022-08-27T07:07:56.000Z,"{'mentions': [{'start': 3, 'end': 19, 'usernam...",NaN,118,0,0,0,1563423048757575683,"[{'type': 'retweeted', 'id': '1563404196421910...",1563423048757575683,NaN,1504854239285329920,Twitter for Android,RT @Merilov72140135: *Special military targets...,everyone,NaN,NaN,NaN
9989,False,2022-08-27T07:07:56.000Z,"{'annotations': [{'start': 71, 'end': 78, 'pro...","[{'domain': {'id': '10', 'name': 'Person', 'de...",0,1,5,0,1563423048484601856,"[{'type': 'replied_to', 'id': '156342223508355...",1563421554989338624,1440629206455513098,930144911684460544,Twitter for iPhone,@AlexArnsy01 @Goodluckkanza @FabrizioRomano Yo...,everyone,NaN,NaN,NaN
9990,False,2022-08-27T07:07:56.000Z,"{'mentions': [{'start': 3, 'end': 15, 'usernam...",NaN,2162,0,0,0,1563423047649873920,"[{'type': 'retweeted', 'id': '1563138745641336...",1563423047649873920,NaN,438164575,Twitter for Android,RT @klllngsleyy: Is anyone else a little scare...,everyone,NaN,NaN,NaN
9991,False,2022-08-27T07:07:56.000Z,"{'annotations': [{'start': 188, 'end': 189, 'p...","[{'domain': {'id': '86', 'name': 'Movie', 'des...",0,0,2,0,1563423047310123009,"[{'type': 'quoted', 'id': '1563211158039175170'}]",1563423047310123009,NaN,18237711,Twitter Web App,"my only comment on this entire ""don't worry da...",everyone,NaN,NaN,NaN
9992,False,2022-08-27T07:07:55.000Z,"{'mentions': [{'start': 0, 'end': 12, 'usernam...",NaN,3,0,6,0,1563423046194843653,"[{'type': 'replied_to', 'id': '156332437342323...",1563324373423235075,1296255404029014017,1526442901877985281,Twitter for iPhone,@Woody2Jenny Oh we notice. Don’t you worry. ht...,everyone,{'media_keys': ['3_1563423040729657344']},NaN,NaN


In [98]:
# Save the data in a .csv file for further analysis
tweets_data.to_csv("tweets_data_1.csv", index=False)
users_data.to_csv("users_data_1.csv", index=False)